In [3]:
#import modules
import pandas as pd
import numpy as np
import Bio
from Bio import Seq
from Bio import SeqIO
import torch
import matplotlib.pyplot as plt
import sys
from torch.utils.data import TensorDataset, DataLoader

In [4]:
#load the arrays into tensors and save them for training and testing the CNN model

X_data = torch.load('/mnt/home/achazing/other/classes/data_science_class/ThermoDrift/tensors/tensor_x.pt') #load tensor to share for training
y_data = torch.load('/mnt/home/achazing/other/classes/data_science_class/ThermoDrift/tensors/tensor_y.pt') #load tensor to share for training



In [24]:
def add_aa_composition_to_tensor(X_tensor):
    #first get the shape of the entire tensor
    tensor_shape = X_tensor.shape
    
    # sum_aa_counts has summed all the 1.0 for every type of aa in a 500 aa sequence
    #this array should have size [n_sequences(i.e. 60k), 25(types of aas)]
    sum_aa_counts = torch.sum(X_tensor, dim=1)
    print('sum_aa_counts: ',sum_aa_counts.shape)
    #assert sum_aa_counts.shape[0] == tensor_shape[0]
    
    # sum the number of amino acid positions that contain data in them to get the length of every sequence
    #this array should have size [n_sequences]
    #lengths_old = torch.sum((sum_aa_counts > 0).long(),dim=1)
    lengths =  torch.sum(sum_aa_counts, dim=1)
    print('lengths: ',lengths.shape)
    
    #unsqueeze the lengths array to make an array of size [n_sequences,1]
    #Then expand this array to have duplicate sequence lengths for all 25 amino acid types
    #this repeat array should have size [n_sequences, 25(types of aas)]
    repeat = lengths.unsqueeze(dim=1).expand(-1,25)
    print('repeat: ',repeat.shape)
    
    #Divide the summed counts of each aa type in a given sequence by the length of that sequence to get percent aa
    #Then unsqueeze to add a dimension at dim=1
    #this divide_tensors array should have size [n_sequences, 1, 25(types of aas)]
    divide_tensors = sum_aa_counts/repeat
    divide_tensors = divide_tensors.unsqueeze(dim=1)
    print('divide_tensors: ',divide_tensors.shape)

    
    #Now concatenate the amino acid fractions of each sequence to the end of the original tensor
    #this new combine_tensors array should have size [n_sequences, 501, 25]
    combine_tensors = torch.cat((X_tensor,divide_tensors),dim=1)
    print('combine_tensors: ',combine_tensors.shape)

    return combine_tensors

In [16]:
X_data_subset = X_data[:1,:,:]

In [17]:
X_data_subset.shape

torch.Size([1, 500, 25])

In [18]:
X_new = add_aa_composition_to_tensor(X_data_subset)

sum_aa_counts:  torch.Size([1, 25])
lengths:  torch.Size([1])
repeat:  torch.Size([1, 25])
divide_tensors:  torch.Size([1, 1, 25])
combine_tensors:  torch.Size([1, 501, 25])


In [19]:
X_new.shape

torch.Size([1, 501, 25])

In [23]:
sum(X_new[0,-1,:])

tensor(1.0000, dtype=torch.float64)

In [ ]:
# sum_aa_counts has summed all the 1.0 for every type of aa in a 500 aa sequence
#this array should have size [n_sequences(i.e. 60k), 25(types of aas)]
sum_aa_counts = torch.sum(X_data, dim=1)
sum_aa_counts.shape

In [ ]:
lengths = torch.sum((sum_aa_counts > 0).long(),dim=1)
lengths.shape

In [ ]:

#repeat = torch.cat([lengths[None]]*25,dim=0)
repeat = lengths.unsqueeze(dim=1).expand(-1,25)
repeat.shape


In [ ]:
divide_tensors = sum_tensor/repeat
divide_tensors = divide_tensors.unsqueeze(dim=1)
divide_tensors.shape

In [ ]:
combine_tensors = torch.cat((X_data_subset,divide_tensors),dim=1)
combine_tensors.shape

In [ ]:
combine_tensors[1,495:501,:]